In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam

from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

This notebook is a blend of what I have learned from other Kaggle Notebooks and external links.

Appropriate external links are placed throughout the notebook and the following notebooks I found helpful are in the list below. 

Chinmay Rane -> https://www.kaggle.com/fuzzywizard/beginners-guide-to-cnn-accuracy-99-7

DATAI -> https://www.kaggle.com/kanncaa1/convolutional-neural-network-cnn-tutorial

Chris Deotte -> https://www.kaggle.com/cdeotte/25-million-images-0-99757-mnist

# Loading Data

In [ ]:
#loading data
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
df = train.copy()
df_test = test.copy()

In [ ]:
#Just having a look at the data
df.describe()

We can see there is one column that has the label and the rest are the image pixels!

In [ ]:
#check for any missing values in train data
df.isnull().any().sum()

In [ ]:
#check for any missing values in test data
df_test.isnull().any().sum()

# Data PreProcessing

In [ ]:
## Setting the seeds for Reproducibility.
seed = 3141
np.random.seed(seed)

### Splitting Train and Test Data

In [ ]:
# X is the pixels and Y is the image labels
X = train.iloc[:,1:]
Y = train.iloc[:,0]

#splitting dataframe using train_test_split
x_train , x_test , y_train , y_test = train_test_split(X, Y , test_size=0.1, random_state=seed)

### Reshaping Images

- We have a 1D vector with 784 pixels and we have to reshape it to (28x28x1) before passing it to the CNN. 

- This is because Keras wants an Extra Dimension in the end for channels. If this had been RGB images, there would have been 3 channels, but as MNIST is gray scale it only uses 1.

In [ ]:
#first param in reshape is number of examples. We can pass -1 here as we want numpy to figure that out by itself

#reshape(examples, height, width, channels)
x_train = x_train.values.reshape(-1, 28, 28, 1)
x_test = x_test.values.reshape(-1, 28, 28, 1)
df_test=df_test.values.reshape(-1,28,28,1)

### Data Augmentation

Data augmentation is super important.  In terms of Images it means we can increase the number of images our model sees. 

This can be acheived by Rotating the Image, Flipping the Image, Zooming the Image, Changing light conditions, Cropping it etc.

Keep in mind doing all these things will not always help the model. For example in our situation a vertical_flip would not be wise as 6's would become 9's and vice-versa.

In [ ]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

### Normalization

Pixel values are often stored as integers in the range 0 to 255, which is the range that int-8 can offer.


Normalization:

1. Sets the value of inputs between 0-1
2. Helps Gradient Descent Converge much faster
3. Brings features to equal level and weightage
4. Helps remove distortians in an image caused by light and shadows 

In [ ]:
#convert values to float as result will be a float. If not done vals are set to zero

x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255
df_test = df_test.astype("float32")/255

In [ ]:
#fitting the ImageDataGenerator we defined above
datagen.fit(x_train)

### One hot encoding labels

The labels are given as integers between 0-9. We need to one hot encode them.

For example 4 looks like this: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

This is done so that we have labels for all the classes, and we can easily carry out the Error/Cost during BackPropogation.



In [ ]:
#notice num_classes is set to 10 as we have 10 different labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print(y_train[0])

# Building CNN Model

For image classification CNN's are the best

NOTE: the hardest part is picking right model by understanding the data rather than by tuning hyperparameters

A larger training dataset will really help CNN accuracy

Steps:

1. Use Sequential Keras API 
2. Add Convolutional Layers - Building blocks of ConvNets and what do the heavy computation
3. Add Pooling Layers - Steps along image - reduces params and decreases likelihood of overfitting
4. Add Batch Normalization Layer - Scales down outliers, and forces NN to not relying too much on a Particular Weight
5. Add Dropout Layer -  Regularization Technique that randomly drops a percentage of neurons to avoid overfitting (usually 20% - 50%)
6. Add Flatten Layer - Flattens the input as a 1D vector
7. Add Output Layer - Units equals number of classes. Sigmoid for Binary Classification, Softmax in case of Multi-Class Classification.
8. Add Dense Layer - Fully connected layer which performs a linear operation on the layer's input

In [ ]:
#Conv2d data_format parameter we use 'channel_last' for imgs

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                 input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compiling Model (configure learning process)

Before training the model we need to make sure we specify how the model will "learn"

1. Specify the Optimizer - The optimizer help us minimize the error function. Examples - RMSprop, Adam, AdaGrad, AdaDelta
2. Specify Loss Function -  For Binary Classification use "binary_crossentropy" and for Multi-class Classification use "categorical_crossentropy"
3. Specify the metrics to evaluate model performance



Great article here to understand the ADAM optimizer -> https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

In [ ]:
#Optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999 )

In [ ]:
#Compiling the model
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

### Learning Rate Decay

Many optimization algorithms have a constant learning rate, which will often not reach a local minima.

To implement Learning Rate Decay we can use either LearningRateScheduler or ReduceLRonPlateau.

- LearningRateScheduler -  takes the step decay function as argument and returns updated learning rates for use in optimzer at every epoch stage.

- ReduceLRonPlateau  - monitors a quantity and if no improvement is seen for a 'patience' number of epochs, then the learning rate is reduced by a factor specified manually.

In [ ]:
#for our case LearningRateScheduler will work great
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [ ]:
#visualizing what the learning rate decay will do to the learning rate through every epoch
decays = [(lambda x: 1e-3 * 0.9 ** x)(x) for x in range(10)]
i=1

for lr in decays:
    print("Epoch " + str(i) +" Learning Rate: " + str(lr))
    i+=1

### Early Stopping Rounds

- I would still like to define an early stopping parameter to ensure that my model stops training once I have reached a point where it is no longer necessary to continue training. This is another way to control overfitting.

- Its important to note that we need to specify a validation dataset in the model to use an early_stopping callback

In [ ]:
#by default this is evaluated on 'val_loss'
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

Since we have the LearningRateScheduler, if we dont use the early stopping callback in this model we see an improvement from 96.4% to about 96.6% accuracy as we get closer and closer to the local minima.

# Fitting the Model

In [ ]:
#defining these prior to model to increase readability and debugging
batch_size = 64
epochs = 50

In [ ]:
# Fit the Model
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size), epochs = epochs, 
                              validation_data = (x_test, y_test), verbose=1, 
                              steps_per_epoch=x_train.shape[0] // batch_size,
                              callbacks = [reduce_lr]) #left out early_stopping parameter as it gets better accuracy

### Evaluating the model

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train','Test'])
plt.grid()
plt.show()

Since the validation_accuracy and the accuracy are really close together we can conclude that we are not overfitting the data.

# Output prediction to CSV file

In [ ]:
pred_digits_test = np.argmax(model.predict(df_test),axis=1)
image_id_test = []

for i in range (len(pred_digits_test)):
    image_id_test.append(i+1)
    
d = {'ImageId':image_id_test,'Label':pred_digits_test}
answer = pd.DataFrame(d)

answer.to_csv('answer.csv',index=False)